In [4]:
#Daily Feature
import pandas as pd
import numpy as np
import datetime as datetime
import pytse_client as tse
from pytse_client import download_client_types_records

In [7]:

#Price data
tickers = tse.download(symbols="فولاد", adjust=True, write_to_csv=True)
df = pd.DataFrame.from_dict(tickers["فولاد"])

#Individual data
if __name__ == '__main__':
    records_dict = download_client_types_records("فولاد")
df1 = pd.DataFrame.from_dict(records_dict["فولاد"])
#Rearrange from past to present with reindex
df1 = df1.iloc[::-1]
df1.index = np.arange(len(df1))

#Concat and align two recieved dataset in the initiate date
df.drop(df.index[0:348], inplace=True)
df_feature = pd.concat([df, df1], axis=1)

df.head()

,date,open,high,low,adjClose,value,volume,count,yesterday,close
348,2008-11-26,19.0,20.0,19.0,20.0,295781378,126348,20,20.0,20.0
349,2008-11-29,19.0,26.0,19.0,20.0,122693235343,38410161,295,20.0,20.0
350,2008-11-30,19.0,20.0,19.0,20.0,233924100,100010,15,20.0,20.0
351,2008-12-01,19.0,20.0,19.0,20.0,300845198,128676,13,20.0,20.0
352,2008-12-02,19.0,19.0,19.0,19.0,66728361,28553,9,20.0,19.0


In [6]:
from daily_features import candel_based_feature

df = candel_based_feature(df_feature)
df

weekly candles features


ValueError: could not convert string to Timestamp

In [ ]:
def wght_based_feature(df):
    print('weight based features')
    
    def value20d_based_func(df):
        return_df = pd.DataFrame({\
            'value_20d':df['value'].rolling(20).mean(), \
            })
        return return_df
    
    df_value20_data = pd.concat([df[['date','value']], value20d_based_func(df)], axis=1)
    
    def wght_based_func(stock_date_group):
        return_df = pd.DataFrame({\
            'value_weight':df['value']/df['value'].sum(), \
            'value_weight20d':df['value_20d']/df['value_20d'].sum(), \
            })
        return return_df

    df_wght_based = value20d_based_func(df)


    df_feature = pd.concat([df_value20_data, df_wght_based], axis=1)
    return df_feature

wght_based_feature(df).head()

In [ ]:
def fwdret_based_func(df):
    print('forward price based features')

    df.index = [pd.Timestamp(item) for item in df['date']]

    ret_nanfilled_df = (np.log(df['close']) - np.log(df['yesterday'])).reindex(\
        pd.date_range(df.index.min(), df.index.max()), fill_value="NaN")
    ret_nanfilled_df.name = None
    

    full_ret2d = pd.DataFrame(ret_nanfilled_df.rolling(2).sum().shift(-1))
    full_ret7d = pd.DataFrame(ret_nanfilled_df.rolling(7).sum().shift(-6))
    full_ret14d = pd.DataFrame(ret_nanfilled_df.rolling(14).sum().shift(-13))
    full_ret21d = pd.DataFrame(ret_nanfilled_df.rolling(21).sum().shift(-20))
    full_ret30d = pd.DataFrame(ret_nanfilled_df.rolling(30).sum().shift(-29))
    full_ret60d = pd.DataFrame(ret_nanfilled_df.rolling(60).sum().shift(-59))
    full_ret90d = pd.DataFrame(ret_nanfilled_df.rolling(90).sum().shift(-89))

    return_df = pd.DataFrame({\
        'ret_fwd2d_log':df[[]].merge(full_ret2d, how='left', left_index=True, right_index=True).values[:,0], \
        'ret_fwd7d_log':df[[]].merge(full_ret7d, how='left', left_index=True, right_index=True).values[:,0], \
        'ret_fwd14d_log':df[[]].merge(full_ret14d, how='left', left_index=True, right_index=True).values[:,0], \
        'ret_fwd21d_log':df[[]].merge(full_ret21d, how='left', left_index=True, right_index=True).values[:,0], \
        'ret_fwd30d_log':df[[]].merge(full_ret30d, how='left', left_index=True, right_index=True).values[:,0], \
        'ret_fwd60d_log':df[[]].merge(full_ret60d, how='left', left_index=True, right_index=True).values[:,0], \
        'ret_fwd90d_log':df[[]].merge(full_ret90d, how='left', left_index=True, right_index=True).values[:,0], \
        'ret_wrt_ind_fwd2d_log':df[[]].merge(full_ret2d, how='left', left_index=True, right_index=True).values[:,0], \
        'ret_wrt_ind_fwd7d_log':df[[]].merge(full_ret7d, how='left', left_index=True, right_index=True).values[:,0], \
        'ret_wrt_ind_fwd14d_log':df[[]].merge(full_ret14d, how='left', left_index=True, right_index=True).values[:,0], \
        'ret_wrt_ind_fwd21d_log':df[[]].merge(full_ret21d, how='left', left_index=True, right_index=True).values[:,0], \
        'ret_wrt_ind_fwd30d_log':df[[]].merge(full_ret30d, how='left', left_index=True, right_index=True).values[:,0], \
        'ret_wrt_ind_fwd60d_log':df[[]].merge(full_ret60d, how='left', left_index=True, right_index=True).values[:,0], \
        'ret_wrt_ind_fwd90d_log':df[[]].merge(full_ret90d, how='left', left_index=True, right_index=True).values[:,0], \
        })
    return return_df

fwdret_based_func(df)

In [ ]:
import jdatetime as jdatetime
import swifter as swifter


def calender_based_feature(stock_daily_data):
    print('calendar based features')
    feature_data = pd.DataFrame([])
    feature_detail = dict()

    # converge
    jdatetime.set_locale('fa_IR')
    def jcal_func(row):
        #geogdate = datetime.datetime.strptime(row['date'], '%Y-%m-%d').date()
        geogdate = row['date'].apply(lambda x: datetime.strptime(x, '%Y%m%d%H'))
        jalalidate = jdatetime.date.fromgregorian(day=geogdate.day,month=geogdate.month,year=geogdate.year)
        return pd.Series([jalalidate.strftime("%Y-%m-%d"), jalalidate.weekday()], index=['jdate', 'jweekday'])

    feature_data = stock_daily_data.swifter.apply(jcal_func, axis=1)
    

    return feature_data

calender_based_feature(df)